## Gather country related information from Healthdata's API

Run the following cells to get all the places Healthdata has information on.

In [14]:
import requests
import json
import pandas as pd
from tqdm import tqdm
import os
from fuzzywuzzy import process

In [15]:
# Your API Secret Key (replace 'YOUR_SECRET_KEY' with the actual key)
api_key = 'p3a88u9bwh8d8npn53g7htra8cxgeh8y'

headers = {
    'Content-Type': 'application/json',
    'Authorization': api_key
}


location_name_dict = {}
for i in tqdm(range(218)):
    location_id = i
    location_url = f"https://api.healthdata.org/sdg/v1/GetLocation?location_id={location_id}"

    response = requests.get(location_url, headers=headers)
    try :
        location_name = response.json()['results'][0]['location_name']
        # append locaiton name and id to dictionnary
        location_name_dict[location_id] = location_name
    except(IndexError):
        print(f"location_id {location_id} does not exist")

  0%|          | 1/218 [00:01<03:37,  1.00s/it]

location_id 0 does not exist


  1%|          | 2/218 [00:02<04:11,  1.16s/it]

location_id 1 does not exist


  1%|▏         | 3/218 [00:03<03:42,  1.04s/it]

...

 89%|████████▊ | 193/218 [03:26<00:24,  1.00it/s]

location_id 192 does not exist


 92%|█████████▏| 200/218 [03:35<00:20,  1.13s/it]

location_id 199 does not exist


100%|██████████| 218/218 [03:54<00:00,  1.07s/it]


In [16]:
location_name_dict

{6: 'China',
 7: "Democratic People's Republic of Korea",
 8: 'Taiwan (Province of China)',
 10: 'Cambodia',
 11: 'Indonesia',
 12: "Lao People's Democratic Republic",
 13: 'Malaysia',
 14: 'Maldives',
 15: 'Myanmar',
 16: 'Philippines',
 17: 'Sri Lanka',
 18: 'Thailand',
 19: 'Timor-Leste',
 20: 'Viet Nam',
 22: 'Fiji',
 23: 'Kiribati',
 24: 'Marshall Islands',
 25: 'Micronesia (Federated States of)',
 26: 'Papua New Guinea',
 27: 'Samoa',
 28: 'Solomon Islands',
 29: 'Tonga',
 30: 'Vanuatu',
 33: 'Armenia',
 34: 'Azerbaijan',
 35: 'Georgia',
 36: 'Kazakhstan',
 37: 'Kyrgyzstan',
 38: 'Mongolia',
 39: 'Tajikistan',
 40: 'Turkmenistan',
 41: 'Uzbekistan',
 43: 'Albania',
 44: 'Bosnia and Herzegovina',
 45: 'Bulgaria',
 46: 'Croatia',
 47: 'Czechia',
 48: 'Hungary',
 49: 'North Macedonia',
 50: 'Montenegro',
 51: 'Poland',
 52: 'Romania',
 53: 'Serbia',
 54: 'Slovakia',
 55: 'Slovenia',
 57: 'Belarus',
 58: 'Estonia',
 59: 'Latvia',
 60: 'Lithuania',
 61: 'Republic of Moldova',
 62: 'Ru

In [17]:
# save the location_name_dict to a json file
with open('location_name_dict.json', 'w') as f:
    json.dump(location_name_dict, f)

### Functions

In [18]:
def get_response(location_id, year=2024):

    base_url = "https://api.healthdata.org/sdg/v1/GetResultsByLocation"

    headers = {
        'Content-Type': 'application/json',
        'Authorization': api_key
    }

    # Construct the full API request URL
    url = f"{base_url}?year={year}&location_id={location_id}"

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    else:
        return response.json()

In [19]:
def construct_csv(location_name, data):
    with open(f"cards/{location_name}.json", "w") as f:
        json.dump(data, f)

    df = pd.DataFrame(data['results'])
    with open(f"cards/{location_name}.csv", "w") as f:
        df.to_csv(f)

In [20]:
def filter_csv(age_group, sex, location):
    df = pd.read_csv(f"cards/{location}.csv")
    df = df[(df['age_group_name'] == age_group) & (df['sex_label'] == sex)]

    df['mean_estimate'] = pd.to_numeric(df['mean_estimate'], errors='coerce')

    ranked_df = df.groupby('indicator_name').agg({'mean_estimate': 'mean'}).reset_index()
    ranked_df = ranked_df.sort_values(by='mean_estimate', ascending=False).reset_index(drop=True)
    
    return ranked_df

In [21]:
def process_csv(df):
    df = df.head(3)
    text_information = "Based on the location, sex and age group you selected, here are the most prevalent diseases from this region:\n"

    for i, row in df.iterrows():
        text_information += f"{i+1}.{row['indicator_name']}\n"

    return text_information

In [22]:
sex_label_dict = {"Male": "Males", "Female": "Females", "Other": "Both sexes"}

age_group_dict = {
    (0, 0): "Neonatal",
    (1, 4): "Under 5",
    (15, 49): "15-49 years"
}

def get_age_group(age):
    for age_range, category in age_group_dict.items():
        if age_range[0] <= age <= age_range[1]:
            return category
    return "Age-standardized"


def get_closest_location(user_input, location_name_dict):
    location_names = list(location_name_dict.values())
    closest_match = process.extractOne(user_input, location_names)
    # closest_match is a tuple (matched_string, score)
    if closest_match[0]:
        return closest_match[0]
    return None

In [23]:
def process_user_entries(age, sex):
    location_name_dict = json.load(open('location_name_dict.json'))
    age_group = get_age_group(age)
    sex_label = sex_label_dict[sex]
    location = get_closest_location(location, location_name_dict)

    return age_group, sex_label, location

## Craft every json and csv for every country

In [32]:
for id, name in tqdm(location_name_dict.items()):
    response = get_response(id)
    if response:
        construct_csv(name, response)

100%|██████████| 185/185 [02:06<00:00,  1.46it/s]


Print informations

In [3]:
for file in os.listdir("cards"):
    if file.endswith(".csv"):
        df = pd.read_csv(f"cards/{file}")

        print("------" + file + "------\n")
        print("\nUnique values for 'age_group_name':")
        print(df['age_group_name'].unique())
        print("\nUnique values for 'indicator_short':")
        print(df['indicator_short'].unique())
        print("\n")

------Djibouti.csv------


Unique values for 'age_group_name':
['All Ages' '15-49 years' 'Under 5' 'Neonatal' 'Age-standardized']

Unique values for 'indicator_short':
['HIV Incid' 'TB Incid' 'Malaria Incid' 'NTD Prev' 'FP Need Met, Mod'
 'UHC Index' 'Under-5 Mort' 'Neonatal Mort' 'Child Stunting' 'Sanitation'
 'Mat Mort Ratio' 'Vaccine Cov DPT3' 'Vaccine Cov MCV2' 'Vaccine Cov PCV'
 'Smoking Prev']


------Solomon Islands.csv------


Unique values for 'age_group_name':
['All Ages' '15-49 years' 'Under 5' 'Neonatal' 'Age-standardized']

Unique values for 'indicator_short':
['HIV Incid' 'TB Incid' 'Malaria Incid' 'NTD Prev' 'FP Need Met, Mod'
 'UHC Index' 'Under-5 Mort' 'Neonatal Mort' 'Child Stunting' 'Sanitation'
 'Mat Mort Ratio' 'Vaccine Cov DPT3' 'Vaccine Cov MCV2' 'Vaccine Cov PCV'
 'Smoking Prev']


------Singapore.csv------


Unique values for 'age_group_name':
['All Ages' '15-49 years' 'Under 5' 'Neonatal' 'Age-standardized']

Unique values for 'indicator_short':
['HIV Incid' '

## Test the function with an example

In [53]:
print(process_csv(filter_csv('All Ages', 'Females', 'Palestine')))


Based on the location, sex and age group you selected, here are the most prevalent diseases from this region:
1.Prevalence of 15 neglected tropical diseases
2.Tuberculosis incidence rate
3.HIV incidence rate

